[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/40--spark/03--rdd-operations.ipynb)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz -O spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [ ]:
!pip install -q pyspark findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("RDD Operations")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

21/12/11 07:28:40 WARN Utils: Your hostname, Jakubs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.8 instead (on interface en0)
21/12/11 07:28:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/11 07:28:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/11 07:28:41 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [4]:
spark

In [2]:
sc = spark.sparkContext

### The filter transformation

In [4]:
!mkdir -p var
!wget https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt -O var/krzyzacy-1.txt

--2021-12-11 07:29:52--  https://wolnelektury.pl/media/book/txt/krzyzacy-tom-pierwszy.txt
Resolving wolnelektury.pl (wolnelektury.pl)... 51.83.143.148
Connecting to wolnelektury.pl (wolnelektury.pl)|51.83.143.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 740054 (723K) [text/plain]
Saving to: ‘var/krzyzacy-1.txt’

var/krzyzacy-1.txt  100%[===================>] 722.71K  --.-KB/s    in 0.07s   

2021-12-11 07:29:52 (10.8 MB/s) - ‘var/krzyzacy-1.txt’ saved [740054/740054]



In [5]:
raw_data = sc.textFile("var/krzyzacy*")

In [6]:
raw_data.count()

15477

In [7]:
sentences = raw_data.filter(lambda x: "" != x)

In [8]:
sentences.count()

7654

### Map transformation

In [9]:
words_batch = sentences.map(lambda x: x.split(" "))

In [10]:
words_batch.take(10)

[['Henryk', 'Sienkiewicz'],
 ['Krzyżacy'],
 ['Tom', 'I'],
 ['ISBN', '978-83-288-2813-1'],
 ['Rozdział', 'pierwszy'],
 ['W',
  'Tyńcu,',
  'w',
  'gospodzie',
  '„Pod',
  'Lutym',
  'Turem”,',
  'należącej',
  'do',
  'opactwa,',
  'siedziało',
  'kilku',
  'ludzi,',
  'słuchając',
  'opowiadania',
  'wojaka',
  'bywalca,',
  'który',
  'z',
  'dalekich',
  'stron',
  'przybywszy,',
  'prawił',
  'im',
  'o',
  'przygodach,',
  'jakich',
  'na',
  'wojnie',
  'i',
  'w',
  'czasie',
  'podróży',
  'doznał.',
  'Człek',
  'był',
  'brodaty,',
  'w',
  'sile',
  'wieku,',
  'pleczysty,',
  'prawie',
  'ogromny,',
  'ale',
  'wychudły;',
  'włosy',
  'nosił',
  'ujęte',
  'w',
  'pątlik,',
  'czyli',
  'w',
  'siatkę',
  'naszywaną',
  'paciorkami;',
  'na',
  'sobie',
  'miał',
  'skórzany',
  'kubrak',
  'z',
  'pręgami',
  'wyciśniętymi',
  'przez',
  'pancerz,',
  'na',
  'nim',
  'pas,',
  'cały',
  'z',
  'miedzianych',
  'klamr;',
  'za',
  'pasem',
  'nóż',
  'w',
  'rogowej',
  'p

### flatMap transformation

In [11]:
words = sentences.flatMap(lambda x: x.split(" "))
words = words.map(lambda x: x.lower())

In [12]:
words_occurence = words.map(lambda word: (word, 1))

In [13]:
wordCounts = words_occurence.reduceByKey(lambda a,b: a + b)

In [14]:
wordCounts.map(lambda x: (x[1], x[0])) \
    .sortByKey(ascending=False) \
    .take(5)

[(9473, 'i'), (8339, '—'), (5262, 'się'), (4782, 'nie'), (4748, 'w')]

In [15]:
!wget https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt -O var/stopwords.txt

--2021-12-11 07:32:31--  https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2016 (2.0K) [text/plain]
Saving to: ‘var/stopwords.txt’

var/stopwords.txt   100%[===================>]   1.97K  --.-KB/s    in 0s      

2021-12-11 07:32:31 (9.11 MB/s) - ‘var/stopwords.txt’ saved [2016/2016]



In [16]:
stop_words = sc.textFile("var/stopwords.txt").collect()

In [17]:
stop_words

['a',
 'aby',
 'ach',
 'acz',
 'aczkolwiek',
 'aj',
 'albo',
 'ale',
 'alez',
 'ależ',
 'ani',
 'az',
 'aż',
 'bardziej',
 'bardzo',
 'beda',
 'bedzie',
 'bez',
 'deda',
 'będą',
 'bede',
 'będę',
 'będzie',
 'bo',
 'bowiem',
 'by',
 'byc',
 'być',
 'byl',
 'byla',
 'byli',
 'bylo',
 'byly',
 'był',
 'była',
 'było',
 'były',
 'bynajmniej',
 'cala',
 'cali',
 'caly',
 'cała',
 'cały',
 'ci',
 'cie',
 'ciebie',
 'cię',
 'co',
 'cokolwiek',
 'cos',
 'coś',
 'czasami',
 'czasem',
 'czemu',
 'czy',
 'czyli',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokad',
 'dokąd',
 'dosc',
 'dość',
 'duzo',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzis',
 'dzisiaj',
 'dziś',
 'gdy',
 'gdyby',
 'gdyz',
 'gdyż',
 'gdzie',
 'gdziekolwiek',
 'gdzies',
 'gdzieś',
 'go',
 'i',
 'ich',
 'ile',
 'im',
 'inna',
 'inne',
 'inny',
 'innych',
 'iz',
 'iż',
 'ja',
 'jak',
 'jakas',
 'jakaś',
 'jakby',
 'jaki',
 'jakichs',
 'jakichś',
 'jakie',
 'jakis',
 'jakiś',
 'jakiz',
 'jakiż',
 'jak

In [18]:
counter = wordCounts.map(lambda x: (x[1], x[0])) \
    .filter(lambda x: x[1] not in stop_words) \
    .sortByKey(ascending=False)

In [19]:
counter.take(15)

[(8339, '—'),
 (616, 'zbyszko'),
 (602, 'zaś'),
 (455, 'rzekł'),
 (415, 'maćko'),
 (360, 'się,'),
 (346, 'począł'),
 (336, 'rzekł:'),
 (329, 'jeno'),
 (316, 'czym'),
 (303, 'chwili'),
 (280, 'de'),
 (260, 'bóg'),
 (249, 'miał'),
 (242, 'zaraz')]

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

### The collect action

Basically it will get all the elements in the RDD into memory for us to work with them. For this reason it has to be used with care, specially when working with large RDDs.

In [20]:
counted_words = counter.collect()

In [21]:
counted_words[:20]

[(8339, '—'),
 (616, 'zbyszko'),
 (602, 'zaś'),
 (455, 'rzekł'),
 (415, 'maćko'),
 (360, 'się,'),
 (346, 'począł'),
 (336, 'rzekł:'),
 (329, 'jeno'),
 (316, 'czym'),
 (303, 'chwili'),
 (280, 'de'),
 (260, 'bóg'),
 (249, 'miał'),
 (242, 'zaraz'),
 (239, 'ni'),
 (236, 'potem'),
 (214, 'stary'),
 (204, 'jurand'),
 (203, 'mógł')]

In [22]:
sc.stop()